In [70]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [37]:
with open('../data/songs.txt', 'r', encoding='utf-8') as f:
    plain_data = f.read()

In [38]:
len(plain_data)

612105

In [39]:
print(plain_data[:500])

Song Name: Tim McGraw
He said the way my blue eyes shined
Put those Georgia stars to shame that night
I said Thats a lie
Just a boy in a Chevy truck
That had a tendency of gettin stuck
On back roads at night
And I was right there beside him all summer long
And then the time we woke up to find that summer gone

But when you think Tim McGraw
I hope you think my favorite song
The one we danced to all night long
The moon like a spotlight on the lake
When you think happiness
I hope you think that lit


In [40]:
# seeing all the unique characters in the data to perform the cleaning
print(''.join(sorted(list(set(plain_data)))))


 !&'(),-./0123456789:?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz®éíïó


In [41]:
# Replace the unwanted characters with the most probable ones

# ® -> ''
# é -> e
# ó -> o
# í -> i
# ï -> i

plain_data = plain_data.replace('®', '')
plain_data = plain_data.replace('é', 'e')
plain_data = plain_data.replace('ó', 'o')
plain_data = plain_data.replace('í', 'i')
plain_data = plain_data.replace('ï', 'i')

In [42]:
chars = sorted(list(set(plain_data)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !&'(),-./0123456789:?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
75


In [43]:
# basic tokenizer
# converting each character to its index in the unique characters array
# Google uses SentencePiece to tokenize
# OpenAI uses tiktoken to tokenize

stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}

encode = lambda s: [stoi[i] for i in s]
decode = lambda s: ''.join([itos[i] for i in s])

In [44]:
encoded_data = encode('hello world')
print("Encoded data is: ", encoded_data)

Encoded data is:  [56, 53, 60, 60, 63, 1, 71, 63, 66, 60, 52]


In [45]:
print("Decoded data is: ", decode(encoded_data))

Decoded data is:  hello world


In [46]:
import torch

In [47]:
# converting the data to a tensor
data = torch.tensor(encode(plain_data), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([612104]) torch.int64
tensor([41, 63, 62, 55,  1, 36, 49, 61, 53, 21,  1, 42, 57, 61,  1, 35, 51, 29,
        66, 49, 71,  0, 30, 53,  1, 67, 49, 57, 52,  1, 68, 56, 53,  1, 71, 49,
        73,  1, 61, 73,  1, 50, 60, 69, 53,  1, 53, 73, 53, 67,  1, 67, 56, 57,
        62, 53, 52,  0, 38, 69, 68,  1, 68, 56, 63, 67, 53,  1, 29, 53, 63, 66,
        55, 57, 49,  1, 67, 68, 49, 66, 67,  1, 68, 63,  1, 67, 56, 49, 61, 53,
         1, 68, 56, 49, 68,  1, 62, 57, 55, 56, 68,  0, 31,  1, 67, 49, 57, 52,
         1, 42, 56, 49, 68, 67,  1, 49,  1, 60, 57, 53,  0, 32, 69, 67, 68,  1,
        49,  1, 50, 63, 73,  1, 57, 62,  1, 49,  1, 25, 56, 53, 70, 73,  1, 68,
        66, 69, 51, 59,  0, 42, 56, 49, 68,  1, 56, 49, 52,  1, 49,  1, 68, 53,
        62, 52, 53, 62, 51, 73,  1, 63, 54,  1, 55, 53, 68, 68, 57, 62,  1, 67,
        68, 69, 51, 59,  0, 37, 62,  1, 50, 49, 51, 59,  1, 66, 63, 49, 52, 67,
         1, 49, 68,  1, 62, 57, 55, 56, 68,  0, 23, 62, 52,  1, 31,  1, 71, 49,
       

In [48]:
# splitting the data in the training and test set
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [49]:
# defining a block size, sometimes also called the context length
# we take block_size+1 because there are 8 examples residing here
block_size = 8
train_data[:block_size+1]

tensor([41, 63, 62, 55,  1, 36, 49, 61, 53])

In [50]:
for i in range(block_size):
    print(f"When the context is: {train_data[:i+1]} the output is: {train_data[i+1]}")

When the context is: tensor([41]) the output is: 63
When the context is: tensor([41, 63]) the output is: 62
When the context is: tensor([41, 63, 62]) the output is: 55
When the context is: tensor([41, 63, 62, 55]) the output is: 1
When the context is: tensor([41, 63, 62, 55,  1]) the output is: 36
When the context is: tensor([41, 63, 62, 55,  1, 36]) the output is: 49
When the context is: tensor([41, 63, 62, 55,  1, 36, 49]) the output is: 61
When the context is: tensor([41, 63, 62, 55,  1, 36, 49, 61]) the output is: 53


In [71]:
torch.manual_seed(1729)
batch_size = 1 # number of sequences processed in parellel
block_size = 8 # number of tokens (context length) to be considered in each sequence

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print("Inputs are:")
print(xb.shape)
print(xb)
print("***"*20)
print("Outputs are:")
print(yb.shape)
print(yb)
print("---"*20)

Inputs are:
torch.Size([1, 8])
tensor([[ 1, 69, 64,  1, 68, 56, 53,  1]])
************************************************************
Outputs are:
torch.Size([1, 8])
tensor([[69, 64,  1, 68, 56, 53,  1, 64]])
------------------------------------------------------------


In [52]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1729)

In [53]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) # (B,T,C) => (batch, time(block), channel (vocab_size))
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # cross entropy needs (B,C,T)
        return logits, loss
    
    def generate(self, idx, max_tokens):
        # idx is (B, T) indeces
        for _ in range(max_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time stamp
            logits = logits[:,-1,:] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled sequence to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [72]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([8, 75])
tensor(4.9138, grad_fn=<NllLossBackward0>)


In [77]:
idx = torch.zeros((1,1), dtype=torch.long, device = device)
print(decode(m.generate(idx, max_tokens=100)[0].tolist()))


Ohaylerer Eyoou hang owin he c san o whans
Teer ahthoy ned m
I acl d f we th myoullig Non
Bowaslll e


In [78]:
# create a torch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [79]:
batch_size = 32

for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')
    
    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.3017683029174805


In [80]:
print(decode(m.generate(idx, max_tokens=300)[0].tolist()))


Yor it you t g Therevevethe cr
Im w buve youxie l ma p p

Manou w myou ng fu y oulint the u kitho bl
Ca taig I thifr fou otle: t at anhowo fr awald ang ndger mingayoanoe aprereld ckn
Thee yo

Fiesthe I I ant bry mererthedathe at tof walplst toes I Ancary as you gaclovened d y th dofike p p uss yoon 
